In [53]:
import json
import re
import requests
import os
import glob
import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing  import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import f1_score
from IPython.core.display import HTML, display
import pymorphy2


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hackabong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [54]:
path = 'res_threads'
all_file = open('train.csv', mode='w', encoding='utf8')
for dirpath, dirnames, filenames in os.walk(path):
    for file in filenames:
        with open(dirpath + "//" + file, mode='r', encoding='utf8') as f:
            all_file.write(f.read())

In [55]:
train_df = pd.read_csv("train.csv", names = ['Text', 'Target'])

train_df.head()

,Text,Target
0,мне 18 лет я учусь в вузе на программистая яв...,0
1,это мои сформировавшиеся взгляды ради которых...,0
2,я веган потому что мне не плевать на окружающ...,0
3,здоровым можно быть и на веганстве,0
4,я за лгбт потому что геи бишки и все небинар...,0


In [56]:
train_df.shape

(10406, 2)

In [57]:
train_df.dropna(axis=0, inplace=True)

In [58]:
train_df.shape

(10404, 2)

In [59]:
train_df['Target'].value_counts()

0    6920
1    3484
Name: Target, dtype: int64

In [60]:
list_corpus = train_df["Text"].tolist()
list_labels = train_df["Target"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.3, 
                                                                                random_state=40)

In [61]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()

    train = tfidf_vectorizer.fit_transform(data)

    return train, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [62]:
clf_tfidf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf_tfidf.fit(X_train_tfidf, y_train)

y_predicted_tfidf = clf_tfidf.predict(X_test_tfidf)

In [63]:
f1_score(y_test, y_predicted_tfidf)

0.6441162458313483

In [64]:
np.mean(y_test == y_predicted_tfidf)

0.7607303010890455

In [65]:
df_res = pd.DataFrame({'Text': X_test, 'Target': y_predicted_tfidf})

df_res.head()

,Text,Target
0,чтобы я в снегурочку нарядился и стишок ему пр...,0
1,в совке более менее могли делать лишь ракеты и...,1
2,вообще непонятно,0
3,антена для радио,0
4,это кпт например просто всякие полезные мело...,0


In [66]:
df_comp = pd.DataFrame({'Text': X_test, 'Target': y_test})

df_comp.head()

,Text,Target
0,чтобы я в снегурочку нарядился и стишок ему пр...,0
1,в совке более менее могли делать лишь ракеты и...,1
2,вообще непонятно,0
3,антена для радио,0
4,это кпт например просто всякие полезные мело...,0


In [150]:
morph = pymorphy2.MorphAnalyzer()
def normalize(x):
    p = morph.parse(x)[0]
    return p.normal_form

input_str = ["Ты дурак"]
input_str = [' '.join(list(map(normalize , input_str[0].split())))]
input_str_test = tfidf_vectorizer.transform(input_str)
result = clf_tfidf.predict(input_str_test)

print("Кибербуллинг" if result[0] == 1 else "Не кибербуллинг")

Не кибербуллинг


In [44]:
input_str_test = tfidf_vectorizer.transform(input_str)

input_str_test

<1x14809 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [45]:
result = clf_tfidf.predict(input_str_test)

result[0]

1

In [433]:
train_df['Text'].duplicated().value_counts()

False    7912
True     2465
Name: Text, dtype: int64

In [18]:
TOKEN_PATTERN = "[а-яё]+"

In [19]:
def tokenize(text):
    return re.findall(TOKEN_PATTERN, text.lower())

docs = [tokenize(text) for text in list_corpus]

In [20]:
docs[0]

['мне',
 'лет',
 'я',
 'учусь',
 'в',
 'вузе',
 'на',
 'программистая',
 'являюсь',
 'веганом',
 'лгбт',
 'активистом',
 'мультикультуралист',
 'феминист',
 'леволибералом',
 'и',
 'научный',
 'атеист']

In [21]:
from collections import *
occurence = Counter()

for doc in docs:
    for token in doc:
        occurence[token] += 1

In [22]:
occurence.most_common(20)

[('и', 2783),
 ('в', 2410),
 ('не', 2386),
 ('на', 1785),
 ('что', 1292),
 ('с', 1104),
 ('а', 1050),
 ('это', 974),
 ('я', 912),
 ('ты', 861),
 ('то', 815),
 ('как', 716),
 ('у', 626),
 ('если', 509),
 ('по', 488),
 ('так', 483),
 ('за', 460),
 ('все', 417),
 ('он', 406),
 ('из', 380)]

In [23]:
stopword_set = set(nltk.corpus.stopwords.words('russian'))

In [24]:
sorted_tokens = sorted(occurence.keys())

In [25]:
sorted_tokens

['а',
 'ааа',
 'ааааа',
 'ааааррряяяяя',
 'ааарряяя',
 'аахахха',
 'абапе',
 'абибасами',
 'аборт',
 'абортво',
 'абортони',
 'абортя',
 'абсолютна',
 'абсолютно',
 'абсолютный',
 'абстрагироваться',
 'абстрактное',
 'абстрактную',
 'абсурда',
 'абу',
 'абы',
 'абьюзер',
 'абьюзера',
 'абьюзераесть',
 'аватарке',
 'аватарки',
 'августинского',
 'аве',
 'авевова',
 'авивова',
 'авито',
 'авой',
 'авроры',
 'австралии',
 'авто',
 'автобамп',
 'автомат',
 'автомате',
 'автоматом',
 'автомобиле',
 'автомобили',
 'автомобиль',
 'авторитетно',
 'авторитетных',
 'авторитетом',
 'авторов',
 'авторучка',
 'авхавхавхахвха',
 'ага',
 'агенства',
 'агенство',
 'агентства',
 'агентстватак',
 'агентстве',
 'агентством',
 'агностиком',
 'агностицизм',
 'агрессивные',
 'агрессивными',
 'агрессии',
 'агрессия',
 'агрессор',
 'агробыдло',
 'агропромышленный',
 'ад',
 'ада',
 'адаптации',
 'адвокатом',
 'адеквата',
 'адекватами',
 'адекватно',
 'адекватности',
 'адекватные',
 'адекватный',
 'адекватных',

In [26]:
import pymorphy2
lemmatizer = pymorphy2.MorphAnalyzer()

In [34]:
lemmatizer_cache = {}

def lemmatize(token):
    if lemmatizer.word_is_known(token):
        if token not in lemmatizer_cache:
            lemmatizer_cache[token] = lemmatizer.parse(token)[0].normal_form
        return lemmatizer_cache[token]
    return token

lemmatized_docs = [[lemmatize(token) for token in text] for text in docs]

cleared_docs = [[token for token in text if token not in stopword_set] for text in lemmatized_docs]

cleared_docs

[['год',
  'учиться',
  'вуз',
  'программистая',
  'являться',
  'веганом',
  'лгбт',
  'активист',
  'мультикультуралист',
  'феминист',
  'леволибералом',
  'научный',
  'атеист'],
 ['это', 'сформироваться', 'взгляд', 'ради', 'который', 'готовый', 'бороться'],
 ['веган',
  'плевать',
  'окружающий',
  'среда',
  'хотеть',
  'причинять',
  'вред',
  'животное'],
 ['здоровый', 'веганстве'],
 ['лгбт', 'гей', 'бишки', 'всё', 'небинарные', 'это', 'зайка'],
 ['любить', 'любой', 'проявление', 'гейства'],
 ['это', 'личный', 'выбор', 'каждый'],
 ['хотеть', 'россия', 'проходить', 'лгбт', 'парад'],
 ['би', 'сексуал'],
 ['почему',
  'пофиг',
  'человек',
  'заниматься',
  'спальня',
  'поддерживать',
  'феминизм',
  'женщина',
  'абсолютный',
  'равный',
  'мужчина'],
 ['век', 'женщина', 'угнетать', 'продолжать', 'сей', 'пора'],
 ['право',
  'женщина',
  'нужно',
  'уравнять',
  'появиться',
  'женщина',
  'учёный',
  'писатель',
  'инженер',
  'который',
  'ранний',
  'давить'],
 ['женщина',
 

In [29]:
cooccurence = Counter()

for doc in docs:
    for i in range(len(doc) - 1):
        cooccurence[(doc[i], doc[i + 1])] += 1

In [30]:
cooccurence.most_common(20)

[(('потому', 'что'), 164),
 (('и', 'не'), 123),
 (('а', 'не'), 102),
 (('я', 'не'), 91),
 (('у', 'меня'), 90),
 (('это', 'не'), 76),
 (('ты', 'не'), 73),
 (('у', 'тебя'), 72),
 (('что', 'то'), 64),
 (('в', 'том'), 63),
 (('он', 'не'), 59),
 (('новый', 'год'), 59),
 (('что', 'ты'), 56),
 (('что', 'он'), 53),
 (('а', 'если'), 52),
 (('так', 'что'), 52),
 (('если', 'ты'), 50),
 (('то', 'что'), 49),
 (('что', 'не'), 47),
 (('не', 'в'), 45)]

In [31]:
N = sum(occurence.values())

N

90315

In [32]:
pmi = Counter()
min_cooccur = 5

for pair in cooccurence:
    if cooccurence[pair] >= min_cooccur:
        pmi[pair] = np.log(cooccurence[pair] * N / occurence[pair[0]] / occurence[pair[1]])

In [33]:
pmi.most_common()

[(('проявление', 'гейства'), 9.801620926132557),
 (('положения', 'расизма'), 9.801620926132557),
 (('крайней', 'мере'), 9.801620926132557),
 (('российской', 'федерации'), 9.801620926132557),
 (('палю', 'годноту'), 9.801620926132557),
 (('коллегами', 'пообщаться'), 9.801620926132557),
 (('воняли', 'ссаками'), 9.801620926132557),
 (('тестовое', 'задание'), 9.801620926132557),
 (('оценивают', 'звездами'), 9.801620926132557),
 (('пришедшим', 'сотрудникам'), 9.801620926132557),
 (('полном', 'серьезе'), 9.801620926132557),
 (('простейшими', 'базовыми'), 9.801620926132557),
 (('базовыми', 'вещами'), 9.801620926132557),
 (('вещами', 'справится'), 9.801620926132557),
 (('социального', 'альфача'), 9.801620926132557),
 (('омежкина', 'рожденного'), 9.801620926132557),
 (('учёные', 'писатели'), 9.619299369338602),
 (('писатели', 'инженеры'), 9.619299369338602),
 (('целыми', 'днями'), 9.619299369338602),
 (('языктатьяна', 'зигинова'), 9.619299369338602),
 (('брачный', 'договор'), 9.619299369338602),

In [35]:
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x, lowercase=False, max_df=0.8, min_df=2,
    norm='l2'
).fit(cleared_docs)

In [42]:
pd.DataFrame(cleared_docs)

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,год,учиться,вуз,программистая,являться,веганом,лгбт,активист,мультикультуралист,феминист,...,None,None,None,None,None,None,None,None,None,None
1,это,сформироваться,взгляд,ради,который,готовый,бороться,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,веган,плевать,окружающий,среда,хотеть,причинять,вред,животное,None,None,...,None,None,None,None,None,None,None,None,None,None
3,здоровый,веганстве,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,лгбт,гей,бишки,всё,небинарные,это,зайка,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,любить,любой,проявление,гейства,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,это,личный,выбор,каждый,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,хотеть,россия,проходить,лгбт,парад,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,би,сексуал,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,почему,пофиг,человек,заниматься,спальня,поддерживать,феминизм,женщина,абсолютный,равный,...,None,None,None,None,None,None,None,None,None,None


In [38]:
X_train_tfidf_lemmatized = vectorizer.transform(cleared_docs)
X_test_tfidf_lemmatized = vectorizer.transform(cleared_docs)

In [39]:
clf_tfidf_lemmatized = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg', 
                         multi_class='multinomial', n_jobs=-1, random_state=40)
clf_tfidf_lemmatized.fit(X_train_tfidf_lemmatized, y_train)

y_predicted_tfidf_lemmatized = clf_tfidf.predict(X_test_tfidf_lemmatized)

ValueError: Found input variables with inconsistent numbers of samples: [10377, 7263]